In [18]:
import pandas as pd
import altair as alt
from datetime import timedelta
import numpy as np 

alt.data_transformers.enable("vegafusion")


DataTransformerRegistry.enable('vegafusion')

This file is for concating the historical cowichan and 2021 onwards data into one file. 

In [19]:
def preprocess_sql(salmon2, cow, output_file):
    # Filter and reset index for Cowichan watershed
    salmon2_cow_temp = salmon2[salmon2['watershed'] == 'cowichan'].reset_index()
    salmon2_cow = salmon2_cow_temp[['date', 'species', 'site', 'count']]
    salmon2_cow = salmon2_cow[salmon2_cow['species'].isin(['co', 'ck'])]

    # Create dummy variables for sites
    site_dummies = pd.get_dummies(salmon2_cow['site'])
    df_expanded = pd.concat([salmon2_cow, site_dummies], axis=1)
    df_expanded = df_expanded.drop('site', axis=1)

    # Standardize species names and concatenate datasets
    cow['species'] = cow['species'].str.lower()
    df_long = pd.concat([cow, df_expanded]).reset_index(drop=True)
    df_long['species'] = df_long['species'].replace('cn', 'ck')

    # Filter and create dummy variables for species
    df_long_filter = df_long[df_long['species'].isin(['co', 'ck'])]
    species_dummies = pd.get_dummies(df_long_filter['species'])
    df_expanded_1 = pd.concat([df_long_filter, species_dummies], axis=1)
    df_expanded_1 = df_expanded_1.drop('species', axis=1)

    # Save the final dataframe to a CSV file
    df_expanded_1.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")


In [20]:
salmon2 = pd.read_csv('data/data_salmon2.csv')  # Replace with the actual path to your salmon2 data
cow = pd.read_csv('data/cowichan_historic.csv')  # Replace with the actual path to your cow data
output_file = 'data/salmon_concat.csv'

preprocess_sql(salmon2, cow, output_file)

Data saved to data/salmon_concat.csv
